<a href="https://colab.research.google.com/github/ji299/DLHW2/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===============================================================
# 0) 依賴（首次執行）
#    - 第一次跑 Colab 時建議整段直接執行
# ===============================================================
!pip -q install --upgrade pip cython wheel
!pip -q install "pycocotools @ git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"
!pip -q install torchmetrics==1.3.0 timm==0.9.12
!pip -q install --quiet "torchvision>=0.18"

# ===============================================================
# 1) 匯入 + NumPy2 相容層
# ===============================================================
import os, json, time, warnings, numpy as np
for d, n in {"float":"float64","int":"int64","bool":"bool_"}.items():
    if not hasattr(np, d): setattr(np, d, getattr(np, n))

from PIL import Image
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models, ops as tv_ops
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from functools import partial
from google.colab import drive
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ 裝置 =", device)

# ===============================================================
# 2) ⚙️ 超參數
# ===============================================================
IMG_SIZE, BATCH = 512, 8
EPOCH_SEG, EPOCH_DET, EPOCH_CLS = 10, 40, 10
LR_SEG, LR_DET, LR_CLS = 1e-3, 5e-4, 1e-4
L_EWC_SEG_STAGE2, L_EWC_SEG_STAGE3, L_EWC_DET_STAGE3 = 2.0, 100.0, 50.0
TH_CONF, TOPK_DET = 0.005, 256
torch.manual_seed(42)

# ===============================================================
# 3) Google Drive & 路徑
# ===============================================================
drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/DLHW2"          # ← 根據需求調整
SAVE_DIR = "/content/drive/MyDrive/DLHW2/models"   # ← 模型儲存目錄
os.makedirs(SAVE_DIR, exist_ok=True)

COCO_TRAIN_IMG  = f"{DATA_DIR}/mini_coco_det/train/images"
COCO_TRAIN_JSON = f"{DATA_DIR}/mini_coco_det/train/annotations/train300.json"
COCO_VAL_IMG    = f"{DATA_DIR}/mini_coco_det/val/images"
COCO_VAL_JSON   = f"{DATA_DIR}/mini_coco_det/val/annotations/val300.json"

VOC_TRAIN_IMG   = f"{DATA_DIR}/mini_voc_seg/train/images"
VOC_TRAIN_MASK  = f"{DATA_DIR}/mini_voc_seg/train/masks"
VOC_VAL_IMG     = f"{DATA_DIR}/mini_voc_seg/val/images"
VOC_VAL_MASK    = f"{DATA_DIR}/mini_voc_seg/val/masks"

CLS_TRAIN_DIR   = f"{DATA_DIR}/mini_imagenette_160/train"
CLS_VAL_DIR     = f"{DATA_DIR}/mini_imagenette_160/val"

# ===============================================================
# 4) Dataset（含 cat2idx 修正）
# ===============================================================
class DetDataset(Dataset):
    def __init__(self, img_dir, ann_file, img_size=512, grid=16, cat2idx=None):
        self.img_dir, self.img_size, self.grid = img_dir, img_size, grid
        with open(ann_file) as f: coco = json.load(f)

        if cat2idx is None:
            used = sorted({a["category_id"] for a in coco["annotations"]})
            self.cat2idx = {cid:i for i,cid in enumerate(used)}
        else:
            self.cat2idx = cat2idx
        self.num_classes = len(self.cat2idx)

        self.id2name = {im["id"]: im["file_name"] for im in coco["images"]}
        self.img_ids = list(self.id2name.keys())

        self.ann = {i: [] for i in self.img_ids}
        for a in coco["annotations"]:
            if a["image_id"] in self.ann and a["category_id"] in self.cat2idx:
                cid = self.cat2idx[a["category_id"]]
                self.ann[a["image_id"]].append((cid, *a["bbox"]))

        self.t = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])

    def __len__(self): return len(self.img_ids)

    def __getitem__(self, idx):
        iid = self.img_ids[idx]
        img = Image.open(os.path.join(self.img_dir, self.id2name[iid])).convert("RGB")
        w0,h0 = img.size
        img = self.t(img)

        sx,sy = self.img_size/w0, self.img_size/h0
        cell  = self.img_size/self.grid
        tgt = torch.zeros((self.grid, self.grid, 1+4+self.num_classes))
        for cid,x,y,w,h in self.ann[iid]:
            cx,cy = (x+w/2)*sx, (y+h/2)*sy
            gw,gh = w*sx, h*sy
            gi,gj = int(cx/cell), int(cy/cell)
            tgt[gj,gi,0] = 1.
            tgt[gj,gi,1:5] = torch.tensor([cx/IMG_SIZE,cy/IMG_SIZE, gw/IMG_SIZE,gh/IMG_SIZE])
            tgt[gj,gi,5+cid] = 1.
        return img, tgt

class SegDataset(Dataset):
    def __init__(self, img_dir, mask_dir, img_size=512):
        self.img_dir, self.mask_dir, self.img_size = img_dir,mask_dir,img_size
        self.imgs = sorted([f for f in os.listdir(img_dir)
                            if f.lower().endswith(("jpg","jpeg","png"))])
        self.t = transforms.Compose([
            transforms.Resize((img_size,img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])
    def __len__(self): return len(self.imgs)
    def _mask(self,fn):
        stem = os.path.splitext(fn)[0]
        for ext in (".png",".jpg",".jpeg",".bmp"):
            p = os.path.join(self.mask_dir, stem+ext)
            if os.path.isfile(p): return p
        raise FileNotFoundError
    def __getitem__(self,idx):
        fn  = self.imgs[idx]
        img = Image.open(os.path.join(self.img_dir,fn)).convert("RGB")
        msk = Image.open(self._mask(fn)).convert("L")
        img = self.t(img)
        msk = msk.resize((self.img_size,self.img_size), Image.NEAREST)
        msk = torch.from_numpy(np.array(msk,np.int64))
        msk[msk>=21] = 0
        return img, msk

# -------- Image Classification Dataset --------
t_cls = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
cls_train = datasets.ImageFolder(CLS_TRAIN_DIR, t_cls)
cls_val   = datasets.ImageFolder(CLS_VAL_DIR , t_cls)

def col(b): return torch.stack([x[0] for x in b]), torch.stack([x[1] for x in b])

det_train = DetDataset(COCO_TRAIN_IMG, COCO_TRAIN_JSON)
det_val   = DetDataset(COCO_VAL_IMG , COCO_VAL_JSON , cat2idx=det_train.cat2idx)
seg_train = SegDataset(VOC_TRAIN_IMG, VOC_TRAIN_MASK)
seg_val   = SegDataset(VOC_VAL_IMG , VOC_VAL_MASK)

det_tr_loader  = DataLoader(det_train,  BATCH, True,  drop_last=True, collate_fn=col)
det_val_loader = DataLoader(det_val,    1,     False, collate_fn=col)
seg_tr_loader  = DataLoader(seg_train,  BATCH, True,  drop_last=True)
seg_val_loader = DataLoader(seg_val,    1,     False)
cls_tr_loader  = DataLoader(cls_train,  BATCH, True,  drop_last=True)
cls_val_loader = DataLoader(cls_val,    1,     False)

# ===============================================================
# 5) 單頭模型定義
# ===============================================================
class Unified(nn.Module):
    def __init__(self, n_det, n_seg, n_cls, grid=16):
        super().__init__()
        self.grid = grid
        mbv3 = models.mobilenet_v3_small(
            weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1)
        self.backbone = mbv3.features
        self.neck = nn.Sequential(
            nn.Conv2d(576,160,1), nn.BatchNorm2d(160), nn.ReLU())
        ch = (n_det+5) + n_seg + n_cls
        self.head = nn.Sequential(
            nn.Conv2d(160,160,3,1,1), nn.ReLU(),
            nn.Conv2d(160,ch,1))
        self.n_det, self.n_seg, self.n_cls = n_det, n_seg, n_cls

    def forward(self, x):
        f = self.neck(self.backbone(x))
        out = self.head(f)
        det, seg, cls = torch.split(out,
                                    [self.n_det+5, 21, self.n_cls], dim=1)
        seg = F.interpolate(seg, (IMG_SIZE,IMG_SIZE),
                            mode='bilinear', align_corners=False)
        cls = cls.mean(dim=[2,3])
        return det, seg, cls

# -------- 全域參數計數工具 --------
def count_parameters(net):
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

# -------- 建立模型並印出參數量 --------
model = Unified(det_train.num_classes, 21, len(cls_train.classes)).to(device)
num_params = count_parameters(model)
print(f"🔢 模型總參數量：{num_params/1e6:.2f} M")

if num_params > 8_000_000:
    print("⚠️ 警告：參數超過 8M，請簡化模型架構")
else:
    print("✔ 模型參數數量符合限制（< 8M）")

print(f"🔢 偵測類別數 = {det_train.num_classes}")

# ===============================================================
# 6) 階段 1：語意分割訓練
# ===============================================================
loss_seg = nn.CrossEntropyLoss()
opt_seg  = torch.optim.AdamW(model.parameters(), lr=LR_SEG)
for ep in range(1, EPOCH_SEG+1):
    model.train()
    tot = 0.
    for img, msk in seg_tr_loader:
        img, msk = img.to(device), msk.to(device)
        _, seg_out, _ = model(img)
        l = loss_seg(seg_out, msk)
        opt_seg.zero_grad()
        l.backward()
        opt_seg.step()
        tot += l.item()
    print(f"[Seg] {ep}/{EPOCH_SEG} loss={tot/len(seg_tr_loader):.4f}")

@torch.no_grad()
def calc_mIoU(loader):
    inter = union = 0
    model.eval()
    for img, msk in loader:
        img, msk = img.to(device), msk.to(device)
        _, seg_out, _ = model(img)
        pred = seg_out.argmax(1)
        inter += (pred == msk).sum().item()
        union += msk.numel()
    return inter / union

mIoU_base = calc_mIoU(seg_val_loader)
print(f"mIoU_base = {mIoU_base:.3f}")

# -------- 計算語意分割的 Fisher 資訊矩陣 --------
fisher_seg, theta_seg = {}, {}
for n, p in model.named_parameters():
    fisher_seg[n] = torch.zeros_like(p)
    theta_seg[n] = p.detach().clone()
for img, msk in seg_tr_loader:
    img, msk = img.to(device), msk.to(device)
    model.zero_grad()
    loss_seg(model(img)[1], msk).backward()
    for n, p in model.named_parameters():
        fisher_seg[n] += p.grad.detach()**2
for n in fisher_seg:
    fisher_seg[n] /= len(seg_tr_loader)

# ===============================================================
# 7) 階段 2：目標偵測 + EWC
# ===============================================================
def split_det(t):
    conf = t[:,:1]
    bbox = t[:,1:5].sigmoid().clamp_min(1e-3)
    cls  = t[:,5:]
    return conf, bbox, cls

loss_conf    = partial(tv_ops.sigmoid_focal_loss, gamma=1.0)
loss_bbox    = nn.SmoothL1Loss(beta=0.1)
loss_cls_det = nn.BCEWithLogitsLoss()
opt_det      = torch.optim.AdamW(model.parameters(), lr=LR_DET)

for ep in range(1, EPOCH_DET+1):
    model.train()
    tot = 0.
    for imgs, tgts in det_tr_loader:
        imgs, tgts = imgs.to(device), tgts.to(device)
        det_out, _, _ = model(imgs)
        conf, bbox, cls = split_det(det_out)

        # 前 10 個 epoch 用 BCE warm-up
        if ep <= 10:
            Lc = nn.BCEWithLogitsLoss()(
                conf, tgts[...,0].unsqueeze(1))
        else:
            Lc = loss_conf(
                conf, tgts[...,0].unsqueeze(1), reduction='mean')

        pos = tgts[...,0] > 0
        if pos.sum():
            Lb = loss_bbox(
                bbox.permute(0,2,3,1)[pos],
                tgts[...,1:5][pos])
            Ld = loss_cls_det(
                cls.permute(0,2,3,1)[pos],
                tgts[...,5:][pos])
        else:
            Lb = Ld = torch.tensor(0., device=device)

        # EWC 正則化項
        ewc = sum(
            (fisher_seg[n] * (p - theta_seg[n])**2).sum()
            for n, p in model.named_parameters()
        )
        loss = Lc + Lb + Ld + L_EWC_SEG_STAGE2 * ewc
        opt_det.zero_grad()
        loss.backward()
        opt_det.step()
        tot += loss.item()
    if ep == 1 or ep % 20 == 0:
        print(f"[Det] {ep}/{EPOCH_DET} loss={tot/len(det_tr_loader):.4f}")

# -------- 計算 mAP_base --------
@torch.no_grad()
def eval_map(metric, loader):
    model.eval()
    for imgs, tgts in loader:
        imgs, tgts = imgs.to(device), tgts.to(device)
        det_out, _, _ = model(imgs)
        conf, bbox, cls = split_det(det_out)

        preds, gts = [], []
        for b in range(imgs.size(0)):
            conf_s = conf[b].sigmoid()[0]
            cls_s  = cls[b].sigmoid()
            max_cls, max_lbl = cls_s.max(0)
            score_map = conf_s * max_cls
            flat = score_map.flatten()
            k = min(TOPK_DET, flat.numel())
            vals, idxs = torch.topk(flat, k)
            ys, xs = idxs//model.grid, idxs%model.grid

            p_boxes, p_scores, p_labels = [], [], []
            for sc, j, i in zip(vals, ys, xs):
                if sc < TH_CONF: continue
                cx, cy, w, h = (bbox[b,:,j,i] * IMG_SIZE).tolist()
                x1, y1 = max(0, cx-w/2), max(0, cy-h/2)
                x2, y2 = min(IMG_SIZE, cx+w/2), min(IMG_SIZE, cy+h/2)
                p_boxes.append([x1,y1,x2,y2])
                p_scores.append(sc.item())
                p_labels.append(max_lbl[j,i].item())
            if p_boxes:
                boxes  = torch.tensor(p_boxes, device=device)
                scores = torch.tensor(p_scores, device=device)
                keep   = tv_ops.nms(boxes, scores, 0.5)
                preds.append({
                    "boxes": boxes[keep],
                    "scores": scores[keep],
                    "labels": torch.tensor(p_labels, device=device)[keep]
                })
            else:
                preds.append({
                    "boxes": torch.tensor([[0,0,1,1]], device=device),
                    "scores": torch.tensor([1e-6], device=device),
                    "labels": torch.tensor([0], device=device)
                })

            # 建立 ground-truth
            g_boxes, g_labels = [], []
            for j in range(model.grid):
                for i in range(model.grid):
                    if tgts[b,j,i,0] == 1:
                        cx, cy, w, h = (tgts[b,j,i,1:5] * IMG_SIZE).tolist()
                        x1, y1 = max(0, cx-w/2), max(0, cy-h/2)
                        x2, y2 = min(IMG_SIZE, cx+w/2), min(IMG_SIZE, cy+h/2)
                        g_boxes.append([x1,y1,x2,y2])
                        g_labels.append(tgts[b,j,i,5:].argmax().item())
            if not g_boxes:
                g_boxes, g_labels = [[0,0,1,1]], [0]
            gts.append({
                "boxes": torch.tensor(g_boxes, device=device),
                "labels": torch.tensor(g_labels, device=device)
            })

        metric.update(preds, gts)

metric_base = MeanAveragePrecision(iou_type='bbox', iou_thresholds=[0.5])
eval_map(metric_base, det_val_loader)
mAP_base = metric_base.compute()['map_50'].item()
print(f"mAP_base = {mAP_base:.4f}")

# -------- 計算偵測的 Fisher 資訊矩陣 --------
fisher_det, theta_det = {}, {}
for n, p in model.named_parameters():
    fisher_det[n] = torch.zeros_like(p)
    theta_det[n] = p.detach().clone()
for imgs, tgts in det_tr_loader:
    imgs, tgts = imgs.to(device), tgts.to(device)
    model.zero_grad()
    Lc = loss_conf(
        split_det(model(imgs)[0])[0],
        tgts[...,0].unsqueeze(1),
        reduction='mean'
    )
    Lc.backward()
    for n, p in model.named_parameters():
        fisher_det[n] += p.grad.detach()**2
for n in fisher_det:
    fisher_det[n] /= len(det_tr_loader)

# ===============================================================
# —— 新增：單任務分類 Baseline 訓練與評估（Top1_base）
# ===============================================================
from torchvision.models import MobileNet_V3_Small_Weights

# 1) 建立單獨分類網路
cls_net = models.mobilenet_v3_small(
    weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1
)
cls_net.classifier[3] = nn.Linear(
    cls_net.classifier[3].in_features,
    len(cls_train.classes)
)
cls_net = cls_net.to(device)

# 2) 單任務分類訓練
opt_cls_base  = torch.optim.AdamW(cls_net.parameters(), lr=LR_CLS)
loss_cls_base = nn.CrossEntropyLoss()
for ep in range(1, EPOCH_CLS+1):
    cls_net.train()
    tot = 0.
    for img, lbl in cls_tr_loader:
        img, lbl = img.to(device), lbl.to(device)
        logits = cls_net(img)
        l = loss_cls_base(logits, lbl)
        opt_cls_base.zero_grad()
        l.backward()
        opt_cls_base.step()
        tot += l.item()
    if ep == 1 or ep % 2 == 0:
        print(f"[Cls Base] {ep}/{EPOCH_CLS} loss={tot/len(cls_tr_loader):.4f}")

@torch.no_grad()
def top1_cls_only(net, loader):
    net.eval()
    correct = total = 0
    for img, lbl in loader:
        img, lbl = img.to(device), lbl.to(device)
        preds = net(img).argmax(1)
        correct += (preds == lbl).sum().item()
        total   += lbl.size(0)
    return correct / total

Top1_base = top1_cls_only(cls_net, cls_val_loader)
print(f"Top1_base = {Top1_base:.3f}")

# ===============================================================
# 8) 階段 3：分類 + EWC
# ===============================================================
loss_cls = nn.CrossEntropyLoss()
opt_cls  = torch.optim.AdamW(model.parameters(), lr=LR_CLS)
for ep in range(1, EPOCH_CLS+1):
    model.train()
    tot = 0.
    for img, lbl in cls_tr_loader:
        img, lbl = img.to(device), lbl.to(device)
        _, _, logit = model(img)
        l = loss_cls(logit, lbl)
        # EWC 正則化：語意分割 & 偵測
        e_s = sum(
            (fisher_seg[n] * (p - theta_seg[n])**2).sum()
            for n, p in model.named_parameters()
        )
        e_d = sum(
            (fisher_det[n] * (p - theta_det[n])**2).sum()
            for n, p in model.named_parameters()
        )
        loss = l + L_EWC_SEG_STAGE3 * e_s + L_EWC_DET_STAGE3 * e_d
        opt_cls.zero_grad()
        loss.backward()
        opt_cls.step()
        tot += l.item()
    if ep == 1 or ep % 2 == 0:
        print(f"[Cls] {ep}/{EPOCH_CLS} loss={tot/len(cls_tr_loader):.4f}")

# ===============================================================
# 9) 最終評估
# ===============================================================
@torch.no_grad()
def top1(loader):
    model.eval()
    ok = tot = 0
    for img, lbl in loader:
        img, lbl = img.to(device), lbl.to(device)
        _, _, logit = model(img)
        ok += (logit.argmax(1) == lbl).sum().item()
        tot += lbl.size(0)
    return ok / tot

metric_final = MeanAveragePrecision(iou_type='bbox', iou_thresholds=[0.5])
eval_map(metric_final, det_val_loader)
mAP_final = metric_final.compute()['map_50'].item()
top1_acc  = top1(cls_val_loader)
mIoU_final= calc_mIoU(seg_val_loader)

print("\n===== FINAL =====")
print(f"Top-1    {top1_acc:.3f} (base {Top1_base:.3f}, drop {(Top1_base-top1_acc)/Top1_base*100:.2f}% )")
print(f"mIoU     {mIoU_final:.3f} (drop {(mIoU_base-mIoU_final)/mIoU_base*100:.2f} %)")
print(f"mAP      {mAP_final:.3f} (drop {(mAP_base-mAP_final)/mAP_base*100:.2f} %)")

# —— 檢查是否符合 Top-1 ≥ Top1_base − 5% 條件
if top1_acc >= Top1_base * 0.95:
    print("✔ Top-1 ≥ Top1_base − 5%")
else:
    print("✘ Top-1 < Top1_base − 5%")

# ===============================================================
# 10) 延遲測試 & 存檔（已改為存到 Google Drive）
# ===============================================================
model.eval()
dum = torch.randn(1,3,IMG_SIZE,IMG_SIZE,device=device)
with torch.no_grad():
    for _ in range(10): _ = model(dum)
    if device.type == "cuda": torch.cuda.synchronize()
    t0 = time.time()
    for _ in range(100): _ = model(dum)
    if device.type == "cuda": torch.cuda.synchronize()
    t1 = time.time()
print(f"🚀 latency {(t1-t0)/100*1000:.2f} ms")

save_path = f"{SAVE_DIR}/your_model.pt"
torch.save(model.state_dict(), save_path)
print(f"✔ 模型已儲存 → {save_path}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'pycocotools' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pycocotools'. Discussion can be found at https://github.com/pypa/pip/issues/6334
Reason for being yanked: <none given>
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.0 

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 116MB/s]


🔢 模型總參數量：1.27 M
✔ 模型參數數量符合限制（< 8M）
🔢 偵測類別數 = 63
[Seg] 1/10 loss=0.3485
[Seg] 2/10 loss=0.1108
[Seg] 3/10 loss=0.0800
[Seg] 4/10 loss=0.0526
[Seg] 5/10 loss=0.0351
[Seg] 6/10 loss=0.0278
[Seg] 7/10 loss=0.0237
[Seg] 8/10 loss=0.0475
[Seg] 9/10 loss=0.0626
[Seg] 10/10 loss=0.0345
mIoU_base = 0.975
[Det] 1/40 loss=0.6481
[Det] 20/40 loss=0.0867
[Det] 40/40 loss=0.0570
mAP_base = 0.0032
[Cls Base] 1/10 loss=2.1775
[Cls Base] 2/10 loss=1.7283
[Cls Base] 4/10 loss=0.8834
[Cls Base] 6/10 loss=0.5004
[Cls Base] 8/10 loss=0.3096
[Cls Base] 10/10 loss=0.2167
Top1_base = 0.800
[Cls] 1/10 loss=2.8546
[Cls] 2/10 loss=2.2172
[Cls] 4/10 loss=1.8514
[Cls] 6/10 loss=1.4952
[Cls] 8/10 loss=1.1631
[Cls] 10/10 loss=0.9318

===== FINAL =====
Top-1    0.667 (base 0.800, drop 16.67% )
mIoU     0.975 (drop 0.01 %)
mAP      0.000 (drop 99.57 %)
✘ Top-1 < Top1_base − 5%
🚀 latency 6.41 ms
✔ 模型已儲存 → /content/drive/MyDrive/DLHW2/models/your_model.pt
